# 營收統計

In [1]:
import pandas as pd
import numpy as np
import sqlite3

def 連接資料庫(db):
    global dfRevenue,dfFin,dfDividend,ids

    # 連接到資料庫【月】 =================================================================
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    sql = '''select * from revenue t'''
    dfRevenue = pd.read_sql(sql,conn)

    # 連接到資料庫【季】
    sql2 = '''select * from financialStatements t'''
    dfFin = pd.read_sql(sql2,conn)

    # 連接到資料庫【年】
    sql3 = '''select * from dividend t'''
    dfDividend = pd.read_sql(sql3,conn) 

    # 連接到資料庫【上市上櫃】
    sqlStock = '''select * from stock t'''
    dfStock = pd.read_sql(sqlStock,conn)
    ids = dfStock['code']
    # ==================================================================================
    conn.close()

def 月營收統計(stocks):
    global dataExport1a

    # 整理【月營收】欄位
    dfRevenue2 = dfRevenue.copy()
    dfRevenue2 = dfRevenue2.drop(columns=['open','high','low','updownYuan','updown'])
    dfRevenue2.columns = ['代碼','月別','收盤','營收(億)','月增(%)','年增(%)','累計營收(億)','累計年增(%)']

    # 【月營收資料】
    dataExport1 = pd.DataFrame(columns = ['代碼','收排','收盤','營收(億)','月增(%)','年增(%)','月別','年前3','累計營收(億)','累計年增(%)','當月營收排名','累計排名','營排'])


    for id in stocks:
        dfRevenue2mask = dfRevenue2['代碼'] == id
        dfRevenue3 = dfRevenue2[dfRevenue2mask]
        dfRevenue3 = dfRevenue3.sort_values(by='月別', ascending=False) # 重整排名

        # 收盤排名,當月營收排名 ========================================================
        dfRevenue3.insert(8,'收排',dfRevenue3['收盤'].rank(ascending=False))
        dfRevenue3.insert(9,'當月營收排名',dfRevenue3['營收(億)'].rank(ascending=False))

        # 當月累計排名 ================================================================
        當月 = dfRevenue3.iloc[0]['月別'].split('/',1) # 去除年份取月份
        當月 = 當月[1]
        當月 = "/" + 當月
        累計 = dfRevenue3['月別'].str.contains(當月,na=False) # 取出所有同期月份，並為同期月份進行排名
        dfRevenue3.insert(10,'累計排名',dfRevenue3[累計]['累計營收(億)'].rank(ascending=False))
        dfRevenue3 = dfRevenue3.fillna(0)
        # 年前3 ======================================================================
        總年數 = []
        for k in range(len(dfRevenue3['月別'])):
            當年 = dfRevenue3.iloc[k]['月別'].split('/',1)
            當年 = 當年[0]
            if 當年 not in 總年數:
                總年數.append(當年)
        排3 = pd.DataFrame(columns = ['代碼','月別','收盤','營收(億)','月增(%)','年增(%)','累計營收(億)','累計年增(%)','收排','當月營收排名','累計排名','營排'])
        for e in range(len(總年數)):
            當年 = 總年數[e]
            前3年 = dfRevenue3['月別'].str.contains(當年,na=False) 
            dfRevenue4 = dfRevenue3[前3年].copy()
            dfRevenue4['營排'] = dfRevenue4['營收(億)'].rank(ascending=False)
            排3 = pd.concat([排3,dfRevenue4],axis=0,ignore_index = True)
        
        # ============================================================================
        
        
        dataExport1 = pd.concat([dataExport1,排3],axis=0,ignore_index = True)
    dataExport1a = dataExport1.loc[:,['代碼','收排','收盤','月增(%)','年增(%)','月別','營排','累計年增(%)','當月營收排名','累計排名']]

def 季財報統計(stocks):
    global dataExport2a
    
    dfFin2 = dfFin.copy()
    # loc可以使用文字，iloc則是使用數字
    dfFin2 = dfFin2.loc[:,['代碼','營業利益率','每股稅後盈餘(元)','稅後淨利年成長率','營業利益年成長率','季度']]

    # 【季財報資料】
    dataExport2 = pd.DataFrame(columns = ['代碼','營業利益率','盈比','盈4','盈單','稅後淨利年成長率','營業利益年成長率','季度','每股稅後盈餘(元)'])

    for id in stocks:
        dfFin2mask = dfFin2['代碼'] == id
        dfFin3 = dfFin2[dfFin2mask]
        dfFin3 = dfFin3.sort_values(by='季度', ascending=False) # 重整排名

        盈餘單 = []
        盈餘4 = []

        for y in range(len(dfFin3['季度'])):
            期別期 = dfFin3.iloc[y]['季度'][4:]

            if 期別期 == 'Q1':
                # 盈單 =================================================
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+1]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q2':
                # 盈單 =================================================
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+2]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q3':
                # 盈單
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+3]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q4':
                # 盈單
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            
        try:
            dfFin3['盈單'] = 盈餘單
        except:
            dfFin3['盈單'] = 0

        try:
            dfFin3['盈4'] = 盈餘4
        except:
            dfFin3['盈4'] = 0

        dfFin3['盈比'] = dfFin3['盈單']*4/dfFin3['盈4']
        dfFin3.replace([np.inf, -np.inf], 0, inplace=True)
        dataExport2 = pd.concat([dataExport2,dfFin3],axis=0,ignore_index = True)
    dataExport2a = dataExport2.iloc[:,:8]
    
def 年營收統計(stocks):
    global dataExport3
    
    # 【年營收資料】
    dataExport3 = pd.DataFrame(columns= ['代碼', '股利發放年度', '股利合計', '盈4', '盈配率'])
    
    for id in stocks:
        # 年報表
        dfDividendmask = dfDividend['代碼'] == id
        dfDividend2 = dfDividend[dfDividendmask]

        # 月報表
        dataExport2a股息年mask = dataExport2a['代碼'] == id
        dataExport2a股息年 = dataExport2a[dataExport2a股息年mask]
        
        年報表盈4 = []

        for l in range(len(dfDividend2['股利發放年度'])):
            
            # 盈4 ==============================================
            股息年 = dfDividend2.iloc[l]['股利發放年度']
            季度年 = str(int(股息年)-1) + 'Q4'
            月報表盈4filter = dataExport2a股息年['季度'] == 季度年
            月報表盈4 = dataExport2a股息年[月報表盈4filter]['盈4']
            盈4資料 = 月報表盈4.to_string(index=False)
            # ==================================================
            
            
            if 月報表盈4.empty: # 如果月報表內容為空則
                年報表盈4.append("")
            else:
                年報表盈4.append(盈4資料)

        dfDividend3 = dfDividend2.copy()

        dfDividend3.insert(3,'盈4',年報表盈4)

        盈配率 = []
        for w in range(len(dfDividend3['盈4'])):
            try:
                盈配率股利 = float(dfDividend3.iloc[w]['股利合計'])
                盈配率盈4 = float(dfDividend3.iloc[w]['盈4'])
                盈配率計算 = 盈配率股利/盈配率盈4
                盈配率.append(盈配率計算)
            except:
                盈配率.append("")
        dfDividend3.insert(4,'盈配率',盈配率)
        
        dataExport3 = pd.concat([dataExport3,dfDividend3],axis=0,ignore_index = True)

def 導出月季年(stockMonth,stockSeason,stockYear):
    Today = pd.Timestamp.today().strftime("%Y%m") # 格式化日期

    stockMonth.to_csv(f"01月營收統計{Today}.csv", encoding="big5hkscs",index=False)
    stockSeason.to_csv(f"02季營收統計{Today}.csv", encoding="big5hkscs",index=False)
    stockYear.to_csv(f"03年營收統計{Today}.csv", encoding="big5hkscs",index=False)
    print('已導出全部資料')


if __name__ == "__main__":
    連接資料庫('goodinfoRevenue.db')
    月營收統計(ids)
    季財報統計(ids)
    年營收統計(ids)
    導出月季年(dataExport1a,dataExport2a,dataExport3)

C:\Users\PAN\AppData\Local\Temp\ipykernel_2520\1458844049.py:47: FutureWarning: Dropping of nuisance columns in Series.rank is deprecated; in a future version this will raise TypeError. Select only valid columns before calling rank.
  dfRevenue3.insert(8,'收排',dfRevenue3['收盤'].rank(ascending=False))
C:\Users\PAN\AppData\Local\Temp\ipykernel_2520\1458844049.py:47: FutureWarning: Dropping of nuisance columns in Series.rank is deprecated; in a future version this will raise TypeError. Select only valid columns before calling rank.
  dfRevenue3.insert(8,'收排',dfRevenue3['收盤'].rank(ascending=False))
C:\Users\PAN\AppData\Local\Temp\ipykernel_2520\1458844049.py:47: FutureWarning: Dropping of nuisance columns in Series.rank is deprecated; in a future version this will raise TypeError. Select only valid columns before calling rank.
  dfRevenue3.insert(8,'收排',dfRevenue3['收盤'].rank(ascending=False))
C:\Users\PAN\AppData\Local\Temp\ipykernel_2520\1458844049.py:47: FutureWarning: Dropping of nuisance

已導出全部資料


In [11]:
abmask = dataExport1a['代碼'] == '6504'
dataExport1a[abmask]

,代碼,收排,收盤,月增(%),年增(%),月別,營排,累計年增(%),當月營收排名,累計排名
56489,6504,118.0,72.9,-16.8,-17.4,2022/11,5.0,-11.6,89.0,11.0
56490,6504,119.0,70.0,14.3,6.67,2022/10,1.0,-10.9,25.0,0.0
56491,6504,120.0,68.8,7.81,2.85,2022/09,4.0,-13.0,67.0,0.0
56492,6504,113.0,85.8,14.0,13.1,2022/08,6.0,-15.0,92.0,0.0
56493,6504,116.5,81.3,-10.4,1.3,2022/07,15.5,-18.1,111.5,0.0
...,...,...,...,...,...,...,...,...,...,...
56604,6504,73.5,154.5,-8.84,5.5,2017/12,5.5,5.63,87.5,0.0
56605,6504,36.5,167.0,2.62,-1.28,2017/11,1.5,5.65,50.5,9.5
56606,6504,36.5,167.0,2.62,-1.28,2017/11,1.5,5.65,50.5,9.5
56607,6504,62.5,158.5,3.87,5.84,2017/10,3.5,6.46,60.5,0.0
